In [1]:
import utils

In [2]:
df = utils.read_csv()

In [3]:
df.head()


,Stock,Year,Sector,Revenue,Gross Profit,R&D Expenses,SG&A Expense,Operating Income,Interest Expense,Earnings before Tax,...,Free Cash Flow growth,Receivables growth,Inventory Growth,Asset Growth,Book Value per Share Growth,Debt Growth,R&D Expense Growth,SG&A Expenses Growth,PRICE VAR [%],Class
0,A,2014,Healthcare,4.048000e+09,1.976000e+09,358000000.0,1.199000e+09,419000000.0,110000000.0,546000000.0,...,-0.4369,-0.3037,-0.4615,0.0121,0.0270,-0.3838,0.0623,0.0127,4.179246,1
1,A,2015,Healthcare,4.038000e+09,2.041000e+09,330000000.0,1.189000e+09,522000000.0,66000000.0,443000000.0,...,-0.2111,-0.0319,-0.0575,-0.3085,-0.2139,-0.0048,-0.0782,-0.0083,13.194947,1
2,A,2016,Healthcare,4.202000e+09,2.197000e+09,329000000.0,1.253000e+09,615000000.0,72000000.0,544000000.0,...,0.5352,0.0413,-0.0148,0.0421,0.0400,0.1505,-0.0030,0.0538,45.350940,1
3,A,2017,Healthcare,4.472000e+09,2.409000e+09,339000000.0,1.229000e+09,841000000.0,79000000.0,803000000.0,...,0.0902,0.1474,0.0788,0.0811,0.1527,0.0562,0.0304,-0.0192,0.717881,1
4,A,2018,Healthcare,4.914000e+09,2.687000e+09,385000000.0,1.374000e+09,928000000.0,75000000.0,946000000.0,...,0.2777,0.0718,0.1096,0.0136,-0.0517,-0.1054,0.1357,0.1180,30.976102,1
